# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233166752156                   -0.50    7.0         
  2   -7.249913618418       -1.78       -1.40    1.0   8.92ms
  3   -7.251274113817       -2.87       -2.06    2.0   11.2ms
  4   -7.251313613496       -4.40       -2.42    2.0   11.5ms
  5   -7.251318016465       -5.36       -2.50    1.0   9.40ms
  6   -7.251337715766       -4.71       -3.11    1.0   9.41ms
  7   -7.251338741179       -5.99       -3.87    1.0   9.50ms
  8   -7.251338785864       -7.35       -3.96    3.0   13.4ms
  9   -7.251338795531       -8.01       -4.29    1.0   9.78ms
 10   -7.251338797940       -8.62       -4.68    2.0   11.1ms
 11   -7.251338798630       -9.16       -5.16    2.0   11.8ms
 12   -7.251338798690      -10.22       -5.37    2.0   12.0ms
 13   -7.251338798700      -11.01       -5.66    1.0   9.65ms
 14   -7.251338798704      -11.31       -6.58    2.0   11.5ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07355208407160295
[ Info: Arnoldi iteration step 2: normres = 0.7218509837991405
[ Info: Arnoldi iteration step 3: normres = 0.6569179871994588
[ Info: Arnoldi iteration step 4: normres = 0.28831175510625545
[ Info: Arnoldi iteration step 5: normres = 0.3560003951709746
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.52e-02, 6.53e-02, 2.16e-01, 2.75e-01, 1.46e-02)
[ Info: Arnoldi iteration step 6: normres = 0.38098169006103466
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.84e-03, 1.01e-01, 3.40e-01, 7.22e-02, 1.05e-01)
[ Info: Arnoldi iteration step 7: normres = 0.06085539944081118
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.48e-04, 8.50e-03, 1.69e-02, 1.74e-02, 4.75e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11112590823705876
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.93e